做一些预处理，并使用 K 近邻算法根据客户数据特征预测客户流失。

In [9]:
from __future__ import print_function
import os
import pandas as pd
data_path = ['dataset']
filepath = os.sep.join(data_path + ['Orange_Telecom_Churn_Data.csv'])
data = pd.read_csv(filepath)#加载数据
data.head(1).T

,0
state,KS
account_length,128
area_code,415
phone_number,382-4657
intl_plan,no
voice_mail_plan,yes
number_vmail_messages,25
total_day_minutes,265.1
total_day_calls,110
total_day_charge,45.07


In [10]:
#state, area code,   phone number这三个特征不方便预测，先丢弃
data.drop(['state', 'area_code', 'phone_number'], axis=1, inplace=True)

In [11]:
data.columns

Index(['account_length', 'intl_plan', 'voice_mail_plan',
       'number_vmail_messages', 'total_day_minutes', 'total_day_calls',
       'total_day_charge', 'total_eve_minutes', 'total_eve_calls',
       'total_eve_charge', 'total_night_minutes', 'total_night_calls',
       'total_night_charge', 'total_intl_minutes', 'total_intl_calls',
       'total_intl_charge', 'number_customer_service_calls', 'churned'],
      dtype='object')

- 请注意，有些列是离散数据，有些是浮点数。这些特征需要使用一些方法进行数字编码。
- 最后，请记住 K 最近邻需要缩放数据

In [12]:
from sklearn.preprocessing import LabelBinarizer

lb = LabelBinarizer()#标签二值化

for col in ['intl_plan', 'voice_mail_plan', 'churned']:
    data[col] = lb.fit_transform(data[col])

In [14]:
data.head()

,account_length,intl_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churned
0,128,0,1,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,0
1,107,0,1,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,0
2,137,0,0,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,0
3,84,1,0,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,0
4,75,1,0,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,0


In [15]:
from sklearn.preprocessing import MinMaxScaler

msc = MinMaxScaler()

data = pd.DataFrame(msc.fit_transform(data),  #   np.array
                    columns=data.columns)

In [16]:
data.head()

,account_length,intl_plan,voice_mail_plan,number_vmail_messages,total_day_minutes,total_day_calls,total_day_charge,total_eve_minutes,total_eve_calls,total_eve_charge,total_night_minutes,total_night_calls,total_night_charge,total_intl_minutes,total_intl_calls,total_intl_charge,number_customer_service_calls,churned
0,0.524793,0.0,1.0,0.480769,0.754196,0.666667,0.754183,0.542755,0.582353,0.542866,0.619494,0.520000,0.619584,0.500,0.15,0.500000,0.111111,0.0
1,0.438017,0.0,1.0,0.500000,0.459744,0.745455,0.459672,0.537531,0.605882,0.537690,0.644051,0.588571,0.644344,0.685,0.15,0.685185,0.111111,0.0
2,0.561983,0.0,0.0,0.000000,0.692461,0.690909,0.692436,0.333242,0.647059,0.333225,0.411646,0.594286,0.411930,0.610,0.25,0.609259,0.000000,0.0
3,0.342975,1.0,0.0,0.000000,0.851778,0.430303,0.851740,0.170195,0.517647,0.170171,0.498481,0.508571,0.498593,0.330,0.35,0.329630,0.222222,0.0
4,0.305785,1.0,0.0,0.000000,0.474253,0.684848,0.474230,0.407754,0.717647,0.407959,0.473165,0.691429,0.473270,0.505,0.15,0.505556,0.333333,0.0


创建两张表，分离特征x(除了churned的所有特征),y(churned)

In [17]:
x_cols = [x for x in data.columns if x != 'churned']

# 切分数据成两个dataframes
X_data = data[x_cols]
y_data = data['churned']

In [ ]:
# X_data = data.copy()
# y_data = X_data.pop('churned')

将具有 k=3 值的 K 最近邻模型拟合到此数据并预测相同数据的结果

In [18]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=3)

knn = knn.fit(X_data, y_data)

y_pred = knn.predict(X_data)

In [19]:
y_pred

array([0., 0., 0., ..., 0., 0., 0.])

还没有讨论测量误差的方法，但准确度很容易理解——被正确预测（正确或错误）的标签的百分比。

In [20]:
def accuracy(real, predict):
    return sum(y_data == y_pred) / float(real.shape[0])

In [21]:
accuracy(y_data, y_pred)

0.9422

使用 n_neighbors=3 再次拟合 K 最近邻模型，但这次使用距离作为权重。

In [39]:
knn = KNeighborsClassifier(n_neighbors=3,weights='distance')

knn = knn.fit(X_data, y_data)

y_pred = knn.predict(X_data)

accuracy(y_data, y_pred)

1.0

使用范围从 1 到 20 的 k (n_neighbors) 值拟合 K 最近邻模型。使用统一权重（默认值）。 
Minkowski 距离 (p) 的系数可以设置为 1 或 2——只要保持一致即可。
将每个拟合使用的准确度和 k 值存储在列表或字典中。

In [41]:
result = []
for i in range(1,21):
    knn = KNeighborsClassifier(n_neighbors=i,p=1)

    knn = knn.fit(X_data, y_data)

    y_pred = knn.predict(X_data)
    #knn.predict_proba
    result.append(accuracy(y_data, y_pred))
result

[1.0,
 0.9286,
 0.9456,
 0.9214,
 0.9356,
 0.9216,
 0.929,
 0.917,
 0.9256,
 0.9126,
 0.921,
 0.9124,
 0.9178,
 0.9096,
 0.9146,
 0.9074,
 0.9126,
 0.9054,
 0.9104,
 0.9022]

In [57]:
knn.predict_proba(X_data.iloc[1:2,:])

array([[0.95, 0.05]])

In [60]:
knn.predict(X_data.iloc[1:2,:])#预测结果 注意shape[,]

array([0.])

value_counts(normalize=False, sort=True, ascending=False, bins=None, dropna=True)

1.normalize : boolean, default False　默认false，如为true，则以百分比的形式显示

2.sort : boolean, default True　默认为true,会对结果进行排序

3.ascending : boolean, default False　默认降序排序

4.bins : integer, 格式(bins=1),意义不是执行计算，而是把它们分成半开放的数据集合，只适用于数字数据

5.dropna : boolean, default True　默认删除na值
 

In [65]:
pd.DataFrame(y_pred).value_counts()

0.0    4778
1.0     222
dtype: int64

In [66]:
pd.DataFrame(y_pred).value_counts(normalize=True)



0.0    0.9556
1.0    0.0444
dtype: float64